In [143]:
from sys import platform
import warnings
import fastf1 as ff1
from fastf1 import plotting
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd

if platform == "win32":
    path = 'C:/Users/olive/GitHub/F1-Fantasy-Predictor/'
elif platform == "darwin":
    path = '~/Documents/GitHub/F1-Fantasy-Predictor/'

warnings.filterwarnings("ignore", category=RuntimeWarning)
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [144]:
# Enable the cache
ff1.Cache.enable_cache(path+'data/cache') 
# ff1.Cache.disabled()

In [145]:
year = 2021
circuit = 5

# Load the session data
session = ff1.get_session(year, circuit, 'FP3')
session.load()
laps = session.load_laps(with_telemetry=True)

core           INFO 	Loading data for Monaco Grand Prix - Practice 3 [v2.3.0]
api            INFO 	No cached data found for driver_info. Loading data...
api            INFO 	Fetching driver list...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for timing_data. Loading data...
api            INFO 	Fetching timing data...
api            INFO 	Parsing timing data...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for timing_app_data. Loading data...
api            INFO 	Fetching timing app data...
api            INFO 	Data has been written to cache!
core           INFO 	Processing timing data...
api            INFO 	No cached data found for session_status_data. Loading data...
api            INFO 	Fetching session status data...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for track_status_data. Loading data...
api            INFO 	Fetching trac

In [146]:
# laps = session.laps
laps = laps[[
    'DriverNumber', 'LapTime', 'LapNumber',
    'Sector1Time', 'Sector2Time', 'Sector3Time',
    'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
    'Compound', 'TyreLife', 'FreshTyre', 'Stint',
    'TrackStatus', 'IsAccurate'
    ]]

laps = laps.rename(columns={'DriverNumber': 'driver','LapTime': 'time','LapNumber': 'lapNum','Sector1Time': 's1','Sector2Time': 's2','Sector3Time': 's3','SpeedI1': 'spI1','SpeedI2': 'spI2','SpeedST': 'spst', 'IsPersonalBest': 'isPB','Compound': 'compound','TyreLife': 'tyrelife','FreshTyre': 'isFresh','TrackStatus': 'trackStatus','IsAccurate': 'isAccurate',})

laps = laps[laps['time'].notnull()]

In [147]:
laps['time'] = laps['time'].apply(lambda x: x.total_seconds())
laps['s1'] = laps['s1'].apply(lambda x: x.total_seconds())
laps['s2'] = laps['s2'].apply(lambda x: x.total_seconds())
laps['s3'] = laps['s3'].apply(lambda x: x.total_seconds())

In [148]:
fastest_laps = laps.query('isPB == True').sort_values('time')
fastest_laps

,driver,time,lapNum,s1,s2,s3,spI1,spI2,SpeedFL,spst,isPB,compound,tyrelife,isFresh,Stint,trackStatus,isAccurate
138,33,71.294,16,18.740,33.516,19.038,222.0,201.0,270.0,280.0,True,SOFT,5.0,True,2,1,True
74,55,71.341,21,18.760,33.585,18.996,221.0,203.0,270.0,278.0,True,SOFT,5.0,True,3,2,True
234,16,71.552,23,18.868,33.613,19.071,221.0,205.0,268.0,279.0,True,SOFT,6.0,False,4,1,True
432,77,71.765,19,18.907,33.652,19.206,221.0,202.0,271.0,277.0,True,SOFT,7.0,False,3,1,True
273,11,71.817,17,18.946,33.714,19.157,219.0,204.0,269.0,281.0,True,SOFT,5.0,True,3,1,True
409,4,71.988,16,19.075,33.767,19.146,221.0,201.0,270.0,277.0,True,SOFT,4.0,True,2,2,True
109,44,72.020,7,18.925,33.740,19.355,221.0,204.0,266.0,277.0,True,SOFT,7.0,True,1,1,True
338,7,72.298,16,19.063,33.975,19.260,219.0,204.0,268.0,277.0,True,SOFT,4.0,True,2,2,True
300,10,72.357,26,19.218,33.739,19.400,220.0,198.0,267.0,280.0,True,SOFT,10.0,False,4,1,True
389,5,72.537,20,19.144,34.051,19.342,217.0,202.0,267.0,275.0,True,SOFT,7.0,False,3,1,True


In [149]:
laps.sort_values('time')

,driver,time,lapNum,s1,s2,s3,spI1,spI2,SpeedFL,spst,isPB,compound,tyrelife,isFresh,Stint,trackStatus,isAccurate
138,33,71.294,16,18.740,33.516,19.038,222.0,201.0,270.0,280.0,True,SOFT,5.0,True,2,1,True
140,33,71.309,18,18.760,33.489,19.060,219.0,203.0,268.0,279.0,False,SOFT,7.0,True,2,1,True
74,55,71.341,21,18.760,33.585,18.996,221.0,203.0,270.0,278.0,True,SOFT,5.0,True,3,2,True
78,55,71.363,25,18.771,33.519,19.073,220.0,203.0,269.0,279.0,False,SOFT,9.0,False,4,1,True
67,55,71.452,14,18.775,33.571,19.106,220.0,207.0,267.0,277.0,False,SOFT,14.0,True,1,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,16,110.264,20,33.863,46.606,29.795,123.0,144.0,268.0,161.0,False,SOFT,3.0,True,3,2,True
134,33,111.668,12,51.650,38.511,21.507,196.0,181.0,264.0,246.0,False,SOFT,1.0,True,2,1,False
68,55,112.269,15,22.221,38.006,52.042,159.0,181.0,NaN,253.0,False,SOFT,15.0,True,1,1,False
62,55,112.623,9,29.630,53.161,29.832,116.0,96.0,262.0,201.0,False,SOFT,9.0,True,1,1,True


In [150]:
laps.sort_values('time').query('compound == "SOFT" & tyrelife <= 3.0').groupby(['driver']).agg({
    'time': 'mean'
}).sort_values('time')

,time
driver,
63,80.384667
10,80.996000
3,81.120500
11,83.028000
14,83.109000
55,83.668500
77,83.951200
7,85.124000
6,85.380333
